### One-hot encoder & decoder

In [1]:
import numpy as np

def onehot_enc(lbl, min_val=0):
  mi = min(lbl)
  enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)
  for i, x in enumerate(lbl):
    enc[i, x - mi] = 1
  return enc

def onehot_dec(enc, mi=0):
  return [np.argmax(e) + mi for e in enc]

### Fungsi Aktivasi sigmoid dan turunannya

In [2]:
def sig(X):
  return [1 / (1 + np.exp(-x)) for x in X]

def sigd(X):
  output = []
  for i, x in enumerate(X):
    s = sig([x])[0]
    output.append(s * (1 - s))
  return output

### BackPropagation (2)

#### a) Fungsi Backpropagation

In [3]:
def bp_fit(X, target, layer_conf, max_epoch, max_error=.1,learn_rate=.1, print_per_epoch=100):
  np.random.seed(1)
  nin = [np.empty(i) for i in layer_conf] 
  n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]
  w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i +1]) for i in range(len(layer_conf) - 1)])
  dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
  d = [np.empty(s) for s in layer_conf[1:]]
  din = [np.empty(s) for s in layer_conf[1:-1]]
  epoch = 0
  mse = 1

  for i in range(0, len(n)-1):
    n[i][-1] = 1
  
  while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
    epoch += 1
    mse = 0
    for r in range(len(X)):
      n[0][:-1] = X[r]
      
      #Forward
      for L in range(1, len(layer_conf)):
        nin[L] = np.dot(n[L-1], w[L-1])
        n[L][:len(nin[L])] = sig(nin[L])
      
      #Backprop (output)
      e = target[r] - n[-1]
      mse += sum(e ** 2)
      d[-1] = e * sigd(nin[-1])
      dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))
      
      #Backprop (selain output)
      for L in range(len(layer_conf) - 1, 1, -1):
        din[L-2] = np.dot(d[L-1],np.transpose(w[L-1][:-1]))
        d[L-2] = din[L-2] * np.array(sigd(nin[L-1]))
        dw[L-2] = (learn_rate * d[L-2]) * n[L-2].reshape((-1, 1))
      
      #Update bobot
      w += dw
    
    mse /= len(X)
        
    if print_per_epoch > -1 and epoch % print_per_epoch == 0:
      print(f'Epoch {epoch}, MSE: {mse}')

  return w, epoch, mse

#### b) Fungsi Testing Backpropagation

In [4]:
def bp_predict(X, w):
  n = [np.empty(len(i)) for i in w]
  nin = [np.empty(len(i[0])) for i in w]
  predict = []
  
  n.append(np.empty(len(w[-1][0])))
  
  #tambahan (nilai input bias)
  for i in range(0, len(n)-1):
    n[i][-1]=1

  for x in X:
    n[0][:-1] = x
    
    for L in range(0, len(w)):
      nin[L] = np.dot(n[L], w[L])
      n[L + 1][:len(nin[L])] = sig(nin[L])
      
    predict.append(n[-1].copy()) 
  
  return predict

#### c) Percobaan Klasifikasi Dataset Iris

In [50]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 5, 3),learn_rate=.1, max_epoch=1000, max_error=.041, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)
print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

C:\Users\WILDAN~1\AppData\Local\Temp/ipykernel_23560/1099056643.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i +1]) for i in range(len(layer_conf) - 1)])
C:\Users\WILDAN~1\AppData\Local\Temp/ipykernel_23560/1099056643.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  w += dw


Epoch 25, MSE: 0.40019741902184314
Epoch 50, MSE: 0.3006932940488159
Epoch 75, MSE: 0.253162485458513
Epoch 100, MSE: 0.18785482392338607
Epoch 125, MSE: 0.12936955636385972
Epoch 150, MSE: 0.09646231410288623
Epoch 175, MSE: 0.07893411853258761
Epoch 200, MSE: 0.06884237603253064
Epoch 225, MSE: 0.06251192781134572
Epoch 250, MSE: 0.05823246577773652
Epoch 275, MSE: 0.05515714947170537
Epoch 300, MSE: 0.0528378911150412
Epoch 325, MSE: 0.051022183853091665
Epoch 350, MSE: 0.049559265780986815
Epoch 375, MSE: 0.04835428515157116
Epoch 400, MSE: 0.04734469107899502
Epoch 425, MSE: 0.046487462619307016
Epoch 450, MSE: 0.04575188704085404
Epoch 475, MSE: 0.04511530920543384
Epoch 500, MSE: 0.04456053723865837
Epoch 525, MSE: 0.04407420543865651
Epoch 550, MSE: 0.04364570856836884
Epoch 575, MSE: 0.04326648734867177
Epoch 600, MSE: 0.04292953575851117
Epoch 625, MSE: 0.042629052080726315
Epoch 650, MSE: 0.042360185461381795
Epoch 675, MSE: 0.04211884750587719
Epoch 700, MSE: 0.041901569233